In [2]:
import scipy.io
import pylab
import numpy as np
import datetime 
import ephem
import random
from scipy import spatial
import matplotlib.pyplot as plt
%matplotlib 
from astropy.table import Table
from astropy import units as unit
from astropy.coordinates import SkyCoord
import pandas as pd
from matplotlib.backends.backend_pdf import PdfPages
import astropy
import astropy.constants as const
from astropy.time import Time
from astropy.coordinates import EarthLocation
import sunpy.coordinates
from datetime import datetime, timezone
from math import atan, atan2, degrees, floor, pi, radians, sin, sqrt
from skyfield.api import earth, JulianDate, now, sun


Using matplotlib backend: MacOSX


In [3]:
# Define num here if you want to process only some parts
num = 1000

In [5]:
# Load the data
data=scipy.io.loadmat('BNS-sources-trial-10saleem.mat')['data'] 
if num is not None:
    data = data[:num]
else:
    num = len(data)
#sourceparameters
DL=data[:,0]  #distance
theta=data[:,1]
phi=data[:,2]

#snr at each detector
snrH=data[:,5]
snrL=data[:,6]
snrV=data[:,7]
snrK=data[:,8]
snrI=data[:,9]

In [6]:

#radius = const.R_earth.to(unit.m)  #Radius of earth in m
earth_rad = const.R_earth.to(unit.m).value  #Radius of earth in m
lat = np.rad2deg(np.pi/2 - theta)
long = np.rad2deg(phi)
distance = DL*unit.Mpc

In [7]:
def net_snr(snrlist):
    return np.sqrt(np.sum(snrlist**2, 0))

#or for networks of 5 and 3 detectors, one can use directly,
snrLHVKI = net_snr(np.array((snrH, snrL, snrV, snrK, snrI)))

snrLHV=net_snr(np.array((snrL, snrH, snrV)))

snrLHVK = net_snr(np.array((snrH, snrL, snrV, snrK)))

snrLHVI = net_snr(np.array((snrH, snrL, snrV, snrI)))

In [8]:
def radectolatlong(Ra,Dec,time): #input in degrees
    
    """
    Take the RA, Dec of GW trigger from the table and convert them to  longitude, latitude
    RA, Dec should be floating point numbers (not astropy.coordinates)
    time should be a astropy.Time.time object
    Output is in floating point numbers (not astropy.quantity or astropy.coordinates)
    """
    ra = np.deg2rad(Ra)
    dec = np.deg2rad(Dec)
    time_sid = (time.sidereal_time('apparent','greenwich')).rad
    lat = dec
    long = ra - time_sid 
    return lat,long #output in radians

def latlongtoradec(Lat,Long,time): #input in degrees
    """
    Take the Lat, Long of GW trigger from the saleem's table and convert them to  RA and DEC.
    """
    Lat = np.deg2rad(Lat)
    Long = np.deg2rad(Long)
    time_sid = (time.sidereal_time('apparent','greenwich')).rad
    
    DEC = Lat
    RA = Long + time_sid
    ra = np.rad2deg(RA) % 360  
    dec = np.rad2deg(DEC)
    return ra,dec #output in degree

def is_saa(lat, longi):
    """Return whether the satellite is in South Atlantic Anamoly or not.
    Hard bounds used from the data of RXTE and assumed that the SAA region is rectangular below the inclination of RXTE.
    
    ra - right ascention of the body in degrees
    dec - declination of the body in degrees
    time - UTC time in datetime format
    """
    # lat, longi = radec2latlong(ra, dec, time)
    #lat, longi = eci_ecef(ra, dec, time)
    lat = np.rad2deg(lat)
    longi = np.rad2deg(longi)
    if -23< lat <= -7 and -90<=longi<=-10:
        return True
    
#     if -90<=longi<=-10:
#         y = 7*longi/80 + 23/8
#         if lat < y:
#             return True
    if -100<=longi<=-35:
        y=14*longi/65+189/13
        if lat<=y:
            return True
    
    if -35<longi<-10:
        y=-14/25*longi-63/5
        if lat<y:
            return True
    
    
    if -10<=longi<=40:
        z = -9*longi/25 -43/5
        if lat<z: 
            return True
	

    if -50<= lat <=-23 and -90<=longi<=40:
        return True
    else: return False 

def getxyz(theta,phi):
    """
    Convert cartesian coordinates into theta phi for ANY FRAME OF REFERENCE
    Theta ranges from 0 to pi i.e angle from z-axis axis, and is a floating point number in RADIANS
    Phi ranges from 0 to 2*pi, and is a floating point number in RADIANS
    x,y z are floating point numbers - and make a unit vector
    """
    theta = np.pi/2 - theta
    x=np.sin(theta)*np.cos(phi)
    y=np.sin(theta)*np.sin(phi)
    z=np.cos(theta)
    return x,y,z
def getnorm(vec):
    """
    Takes in the vector and get its norm and returns unit vector
    """
    norm = np.linalg.norm(vec)
    #print (norm)
    return vec/norm

In [9]:
# Create a function that reads the data file and returns whatever table you want
"""  Code replaced by next cell
df = pd.read_csv('Events(triggers).csv')
t1 = df.sort_values(by=['Time in isot'])
t2 = t1.reset_index(drop=True)   #resetting the index 
ra = t2['RA in degree']   #in degree
dec = t2['DEC in degree']
ht = t2['Time in jd']
homotime = Time(ht,format='jd')
source_ra = np.empty(num,dtype=float)
source_dec = np.empty(num,dtype=float)

for i in range(num):
    source_ra[i] = ra[i]
    source_dec[i] = dec[i]
    """

"""
source_ra and source_dec are for removing the unnecessary index in ra and dec.
"""

'\nsource_ra and source_dec are for removing the unnecessary index in ra and dec.\n'

In [10]:
source_table = Table.read("Events(triggers).csv", format='ascii.csv', names=("ID", "RA", "Dec", "Tiso", "JD"))
# Truncate table as needed
source_table = source_table[:num]

source_table.sort("JD")
homotime = Time(source_table["JD"], format='jd')

In [11]:
source_table

ID,RA,Dec,Tiso,JD
int64,float64,float64,str23,float64
623,166.0840774226143,47.688944896871476,2022-01-01T04:45:49.335,2459580.698487678
393,242.62074479324016,-37.14297208642664,2022-01-01T08:11:20.783,2459580.8412127704
166,179.7587860893109,-35.37594873544909,2022-01-01T09:04:01.695,2459580.877797398
164,182.33749163464861,57.27139973411734,2022-01-02T00:00:13.778,2459581.5001594718
523,42.824067479196856,-46.83252299933426,2022-01-02T02:04:14.993,2459581.586284646
458,153.21593867724164,4.2140449039973955,2022-01-02T11:30:01.427,2459581.979183188
204,35.83725396912968,-44.96903743808915,2022-01-02T16:00:17.202,2459582.166865759
522,214.47401981353718,10.365692687193855,2022-01-03T02:45:25.989,2459582.6148841293
450,191.63541074022845,7.733155709359769,2022-01-03T03:18:18.859,2459582.6377182733


In [12]:
sunloc = astropy.coordinates.get_sun(homotime)   # Gives the sun's location 

In [13]:
sunloc

<SkyCoord (GCRS: obstime=[2459580.69848768 2459580.84121277 2459580.8777974  2459581.50015947
 2459581.58628465 2459581.97918319 2459582.16686576 2459582.61488413
 2459582.63771827 2459583.2935744  2459583.33577848 2459583.36970178
 2459583.5833504  2459583.97144224 2459584.52740772 2459584.55436387
 2459584.74726879 2459584.89343552 2459585.28222365 2459585.99857739
 2459586.18504759 2459586.70907557 2459586.75906514 2459587.13354595
 2459587.19773352 2459587.30079796 2459589.00611003 2459589.2075408
 2459589.38952183 2459589.58846275 2459589.70088203 2459589.92580533
 2459591.26175745 2459591.33338926 2459591.54788235 2459591.91413946
 2459591.96691916 2459592.23772428 2459592.62670245 2459593.30511265
 2459594.6525194  2459594.67697023 2459595.07749539 2459595.24117913
 2459595.2708858  2459595.34112655 2459596.19287979 2459598.13566403
 2459598.52489316 2459598.833312   2459599.04906711 2459599.63320968
 2459599.64283762 2459599.87595445 2459599.9342028  2459600.09788042
 2459600.3

In [14]:
sunRA = sunloc.ra.deg  #sun's RA in earth's frame in degrees
sunDEC = sunloc.dec.deg #sun's DEC in earth's frame in degrees
#sunDIST = sunloc.distance.AU*1.496e11 #sun distance from earth in m

In [15]:
"""
Given below is the TLE format data for the satellite, i.e Astrosat. 
sublat and sublong are the latitude and longitude of the satellite at any instant.
alt1 and alt2 are altitude of sat above the earth's surface.
"""
line0 = 'ASTROSAT' #Daksha
line1 = '1 40930U 15052A   22302.17850942  .00000772  00000-0  19439-4 0  9990'
line2 = '2 40930   5.9975   0.5159 0009292 339.1937  20.7998 14.76165238112787'
line3 = '2 40930   5.9975   0.5159 0009292 339.1937 200.7998 14.76165238112787'
sublat1 = np.empty(num,dtype=np.float)  
sublong1 = np.empty(num,dtype=np.float)  
sublat2 = np.empty(num,dtype=np.float)  
sublong2 = np.empty(num,dtype=np.float)

"""earth_ra and earth_dec are ra and dec of earth in refrence frame of satellite."""

#earth_ra1 = np.empty(num,dtype=np.float)
#earth_dec1 = np.empty(num,dtype=np.float)
elevation1 = np.empty(num,dtype=np.float)  # its elevation of 1st daksha satellite
#earth_ra2 = np.empty(num,dtype=np.float)
#earth_dec2 = np.empty(num,dtype=np.float)
elevation2 = np.empty(num,dtype=np.float)

# line0 = 'SWIFT'#BAT
# line1 = '1 28485U 04047A   22302.18405080  .00001301  00000-0  60140-4 0  9997'
# line2 = '2 28485  20.5572 277.8742 0011565 130.4507 229.6936 15.03911323708409'
# line0 = 'FGRST (GLAST)'#Fermi
# line1 = '1 33053U 08029A   22100.45970249  .00000511  00000-0  12538-4 0  9998'
# line2 = '2 33053  25.5819 140.4060 0012690 120.1881 239.9912 15.11069703542313'
astrosat1 = ephem.readtle(line0,line1,line2)
astrosat2 = ephem.readtle(line0,line1,line3)

sat_daksha1 = []
sat_daksha2 = []
for i in range(num):
    time = homotime[i].iso
    astrosat1.compute(time)
    sat_daksha1.append(astrosat1)
    astrosat2.compute(time)
    sat_daksha2.append(astrosat2)

"""
for i in range(num):
    time = homotime[i].iso
    astrosat1.compute(time)
    astrosat2.compute(time)
    sublat1[i] = (astrosat1.sublat)          #sublat in radians
    sublong1[i] = (astrosat1.sublong)        #sublong in radian
    sublat2[i] = (astrosat2.sublat)          #sublat in radians
    sublong2[i] = (astrosat2.sublong)
    #earth_ra1[i] = (np.rad2deg(astrosat1.g_ra) + 180 ) % 360
    #earth_dec1[i] = - np.rad2deg(astrosat1.g_dec)
    elevation1[i] = astrosat1.elevation
    #earth_ra2[i] = (np.rad2deg(astrosat2.g_ra) + 180 ) % 360
    #earth_dec2[i] = - np.rad2deg(astrosat2.g_dec)
    elevation2[i] = astrosat2.elevation
"""

'\nfor i in range(num):\n    time = homotime[i].iso\n    astrosat1.compute(time)\n    astrosat2.compute(time)\n    sublat1[i] = (astrosat1.sublat)          #sublat in radians\n    sublong1[i] = (astrosat1.sublong)        #sublong in radian\n    sublat2[i] = (astrosat2.sublat)          #sublat in radians\n    sublong2[i] = (astrosat2.sublong)\n    #earth_ra1[i] = (np.rad2deg(astrosat1.g_ra) + 180 ) % 360\n    #earth_dec1[i] = - np.rad2deg(astrosat1.g_dec)\n    elevation1[i] = astrosat1.elevation\n    #earth_ra2[i] = (np.rad2deg(astrosat2.g_ra) + 180 ) % 360\n    #earth_dec2[i] = - np.rad2deg(astrosat2.g_dec)\n    elevation2[i] = astrosat2.elevation\n'

In [16]:
"""Satellite RA and DEC"""
sat_ra = (sunRA+180) % 360  #RA is in degrees
sat_dec = -sunDEC   #DEC is in degrees

In [17]:
"""
Distance is 1 m as we can get unit vector.
c and event are the SkyCoord object for the satellite and the sources, respectively.
sv is the vector from sun to earth/satellite in ICRS frame. 
Similarly, trig is the vector from earth/satellite to source.
"""
c = SkyCoord(ra=sat_ra*unit.degree, dec=sat_dec*unit.degree, distance=1*unit.m,frame='gcrs')
sv = np.transpose(np.array([c.cartesian.x.value, c.cartesian.y.value, c.cartesian.z.value]))

event = SkyCoord(ra=source_table["RA"]*unit.degree,dec=source_table["Dec"]*unit.degree,distance=1*unit.m,frame='gcrs')
trig = np.transpose(np.array([event.cartesian.x.value, event.cartesian.y.value, event.cartesian.z.value]))

In [18]:
"""
Define random luminosity of sources b/w 10^-7 and 10^-6. """
lumino = 10**np.random.uniform(-7, -6, num) * (DL / 200.)**2


In [25]:
%%time
earth_ra = np.zeros(len(sat_daksha1))
earth_dec = np.zeros(len(sat_daksha1))
sat_elv = np.zeros(len(sat_daksha1))
for c_sat, sat in enumerate(sat_daksha1):
    earth_ra[c_sat] = sat_daksha1[c_sat].g_ra
    earth_dec[c_sat] = sat_daksha1[c_sat].g_dec
    sat_elv[c_sat] = sat_daksha1[c_sat].elevation

earthcoords = SkyCoord(earth_ra*unit.rad + np.pi*unit.rad, earth_dec*unit.rad)
sourcecoords = SkyCoord(source_table["RA"]*unit.deg, source_table["Dec"]*unit.deg)
eclipse_angle = np.arcsin(const.R_earth/(const.R_earth+sat_elv*unit.m))
sep = earthcoords.separation(sourcecoords)
is_visible_bool = (sep > eclipse_angle)

CPU times: user 6.58 ms, sys: 1.37 ms, total: 7.95 ms
Wall time: 6.52 ms


In [42]:
%%time
earth_ra2 = np.zeros(len(sat_daksha2))
earth_dec2 = np.zeros(len(sat_daksha2))
sat_elv2 = np.zeros(len(sat_daksha2))
for c_sat, sat in enumerate(sat_daksha2):
    earth_ra2[c_sat] = sat_daksha2[c_sat].g_ra
    earth_dec2[c_sat] = sat_daksha2[c_sat].g_dec
    sat_elv2[c_sat] = sat_daksha2[c_sat].elevation

earthcoords2 = SkyCoord(earth_ra2*unit.rad + np.pi*unit.rad, earth_dec2*unit.rad)
sourcecoords2 = SkyCoord(source_table["RA"]*unit.deg, source_table["Dec"]*unit.deg)
eclipse_angle2 = np.arcsin(const.R_earth/(const.R_earth+sat_elv2*unit.m))
sep2 = earthcoords2.separation(sourcecoords2)
is_visible_bool2 = (sep2 > eclipse_angle2)

CPU times: user 11.5 ms, sys: 2.43 ms, total: 13.9 ms
Wall time: 12.1 ms


In [26]:
def is_visible(trig_ra, trig_dec, satephem):
    """
    Check for the visibility of source, i.e whethear the source is block by earth or not.
    eclipse angle is the minimum a1ngle below which the source(trigger) will not be seen from satellite frame.
    earth_ra and earth_dec are ra and dec of earth in refrence frame of satellite.
    trig_ra and trig_dec are ra and dec of trigger.
    And the ra and dec of trig will be same in both frames, i.e in satellite and earth as they are too far away.
    sep is the angular distance between trig and esrth in satellite frame.
    satephem.g_ra and satephem.g_dec return angle in radians.
    """
    #time = homotime.iso
    #satephem.compute(time)
    eclipse_angle = np.arcsin(const.R_earth.to(unit.m).value/(const.R_earth.to(unit.m).value+satephem.elevation))*unit.rad
    earth_ra = (np.rad2deg(satephem.g_ra) + 180 ) % 360
    earth_dec = - np.rad2deg(satephem.g_dec)
    c1 = SkyCoord(earth_ra*unit.deg,earth_dec*unit.deg)
    c2 = SkyCoord(trig_ra*unit.deg,trig_dec*unit.deg)
    sep = c1.separation(c2)
    return sep > eclipse_angle
    #if sep.value > eclipse_angle.to(unit.deg).value:    # to() function is used here to convert rad to deg.
    #    return True
    #else:  
    #    return False

In [27]:
%time is_visible(source_table["RA"][208], source_table["Dec"][208], sat_daksha1[208])

CPU times: user 3.59 ms, sys: 4 µs, total: 3.6 ms
Wall time: 3.61 ms


False

In [28]:
def transform_antisun(astrosat):   
    """Transforming coordinate system to satellite's frame for anti-sun orbit. 
zaxis is the axis from sun to earth and is equal to sv which unit vector.
satvecold is the unit vector from earth to satellite. This unit vector when intersect with satellite, 
the coordinates of that point will be the distance from center of earth times the unit vector.
(satx,saty,satz) are coordinate of the above described point. Now the equation of plane which is perpendicular
to the sv vector is given by sv[0]*(x-satx) + sv[1](y-saty) + sv[2](z-satz) = 0. 
(This is a plane with normal vector sv and contain the point [satx,saty,satz] on it).
The intersection of this plane with the equatorial plane i.e z=0 gives the line sv[0]*(x-satx) + sv[1](y-saty) = 0.
i.e. sv[0]*x + sv[1]*y = sv[0]*satx + sv[1]*saty.

trigvec is the trigger unit vector. yprojtrig and xprojtrig are the the projection vector on x-y plane in satellite
coordinate frame and are used to determine the theta and phi of source.
theta is angle from z-axis and phi is angle in xy-plane from x-axis.
"""
    thetatransarray = np.empty(num,dtype=float)
    phitransarray = np.empty(num,dtype=float)
    #thetatransarray2 = np.empty(num,dtype=float)
    #phitransarray2 = np.empty(num,dtype=float)
    for i in range(num):
        if i%(int((num)/10))==0:print(int(i*100/(num)), '% completed', end='\r')
        zaxis = sv[i]
        #xaxis = getnorm(np.cross(zaxis,satvecold))
        taninv = np.arctan(sv[i][0]/sv[i][1])
        xaxis = [-np.cos(taninv),np.sin(taninv),0]
#        yaxis = getnorm(np.cross(zaxis,xaxis))
        yaxis = np.cross(zaxis,xaxis)
        trigvec = trig[i]

        thetatrans = np.arccos(np.dot(trigvec,zaxis))
        yprojtrig = np.linalg.norm(trigvec)*np.dot(trigvec,yaxis)
        xprojtrig = np.linalg.norm(trigvec)*np.dot(trigvec,xaxis)
        phitrans = np.arctan2(yprojtrig, xprojtrig)
        thetatransarray[i] = thetatrans
        phitransarray[i] = phitrans
    return thetatransarray,phitransarray

#theta1_antisun,phi1_antisun = transform_antisun(astrosat1)
#theta2_antisun,phi2_antisun = transform_antisun(astrosat2)

In [29]:
sd1 = sat_daksha1[0]

## Test cases using 3D plots

In [30]:
"""
to obtain the cartesian coordinate of earth in the satellite frame."""
"""
It gives the position of earth w.r.t satellite. Returns the position in cartesian coordinates."""
def earthpos_wrtsat(time,satephem):
    t = time.iso
    satephem.compute(t)
    earth_ra = (np.rad2deg(satephem.g_ra) + 180 ) % 360
    earth_dec = - np.rad2deg(satephem.g_dec)
    cart = SkyCoord(ra = earth_ra*unit.deg, dec = earth_dec*unit.deg,frame='gcrs')
    return cart.cartesian.x.value,cart.cartesian.y.value,cart.cartesian.z.value

In [31]:
"""
used for plotting the 3D plot."""
"""n is the nth source. d is used to find the cartesian coordinates of source w.r.t earth or satellite as the 
source is too far away."""
n=0

taninv = np.arctan(sv[n][0]/sv[n][1])
satx = getnorm([-np.cos(taninv),np.sin(taninv),0])
satz = sv[n]
saty = getnorm(np.cross(satz,satx))
d = SkyCoord(ra=source_table["RA"][n]*unit.degree,dec=source_table["Dec"][n]*unit.degree,distance=1*unit.m)
trigvec = [d.cartesian.x.value, d.cartesian.y.value, d.cartesian.z.value]
earthvec = earthpos_wrtsat(homotime[n],astrosat1)
is_visible(source_table["RA"][n], source_table["Dec"][n], sat_daksha1[n])

True

In [230]:
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
%matplotlib 
import numpy as np
from itertools import product, combinations


fig = plt.figure()
ax = fig.gca(projection='3d')
ax.set_aspect("equal")

# draw cube
r = [-0.05, 0.05]
for s, e in combinations(np.array(list(product(r, r, r))), 2):
    if np.sum(np.abs(s-e)) == r[1]-r[0]:
        ax.plot3D(*zip(s, e), color="r")

#earth
u, v = np.mgrid[0:2*np.pi:200j, 0:np.pi:200j]
scale=0.75
x = scale*np.cos(u)*np.sin(v) + earthvec[0]
y = scale*np.sin(u)*np.sin(v) + earthvec[1]
z = scale*np.cos(v) + earthvec[2]
ax.plot_wireframe(x, y, z, color="b")


# draw a point
ax.scatter([0], [0], [0], color="g", s=100)
ax.scatter(1.5*trigvec[0],1.5*trigvec[1],1.5*trigvec[2],color='black',s=10)

# draw a vector
from matplotlib.patches import FancyArrowPatch
from mpl_toolkits.mplot3d import proj3d


class Arrow3D(FancyArrowPatch):

    def __init__(self, xs, ys, zs, *args, **kwargs):
        FancyArrowPatch.__init__(self, (0, 0), (0, 0), *args, **kwargs)
        self._verts3d = xs, ys, zs

    def draw(self, renderer):
        xs3d, ys3d, zs3d = self._verts3d
        xs, ys, zs = proj3d.proj_transform(xs3d, ys3d, zs3d, renderer.M)
        self.set_positions((xs[0], ys[0]), (xs[1], ys[1]))
        FancyArrowPatch.draw(self, renderer)
s1 = Arrow3D([0,2*trigvec[0]],[0,2*trigvec[1]],[0,2*trigvec[2]], mutation_scale=20,
            lw=1, arrowstyle="-|>", color="k")

e = Arrow3D([0,earthvec[0]],[0,earthvec[1]],[0,earthvec[2]], mutation_scale=20,
            lw=1, arrowstyle="-|>", color="k")

sun = Arrow3D([0,-2*satz[0]],[0,-2*satz[1]],[0,-2*satz[2]], mutation_scale=20,
            lw=1, arrowstyle="-|>", color="k")

ax.quiver(0,0,0, satx[0], satx[1], satx[2] , color='blue')
ax.quiver(0,0,0, saty[0], saty[1], saty[2], color='green')
ax.quiver(0,0,0, satz[0],satz[1],satz[2], color='red')
ax.text3D(satx[0], satx[1], satx[2] , 'X')
ax.text3D(saty[0], saty[1], saty[2], 'Y')
ax.text3D(satz[0],satz[1],satz[2], 'Z')
ax.text3D(1.5*trigvec[0], 1.5*trigvec[1], 1.5*trigvec[2] , 'Source')
ax.text3D(-2*satz[0],-2*satz[1],-2*satz[2], 'Sun')
ax.text3D(earthvec[0],earthvec[1],earthvec[2], 'Earth')
ax.set_xlim(-1, 1)
ax.set_ylim(-1, 1)
ax.set_zlim(-1, 1)
ax.add_artist(e)
ax.add_artist(s1)
ax.add_artist(sun)
plt.show()


Using matplotlib backend: MacOSX


## Sensitivity Part

In [32]:
sensfile = np.loadtxt('sensitivitydata_20det.csv', delimiter=',',skiprows=1)

In [33]:
"""
thetadeg1, thetadeg2, phideg1 and phideg2 are the angles in degree.
theta1 is from 0 to np.pi rad and phi1 is from 0 to 2*np.pi rad.
thetadeg1 is from 0 to 180 deg and phideg1 is from 0 to 360 deg."""

theta1,phi1 = transform_antisun('astrosat1')   
#theta2,phi2 = transform_antisun('astrosat2')
thetadeg1 = np.rad2deg(theta1) 
#thetadeg2 = np.rad2deg(theta2)
phideg1 = np.rad2deg(phi1 % 2*np.pi)
#phideg2 = np.rad2deg(phi2)
newphi = phi1 % np.pi

In [34]:
sensarray = np.empty(len(sensfile),dtype=float)
sensitivitydata1 = np.empty(num,dtype=float)
sensitivitydata2 = []
sensfilethetaphi = []
for i in range(len(sensfile)):
    sensfilethetaphi.append((sensfile[i][0], sensfile[i][1]))
    sensarray[i] = sensfile[i][3]

In [35]:
"""
sens_source1 is the SkyCoord class for the theta-phi calculated from the code and sens_sat is the SkyCoord class 
for previously calculated theta-phi from the projection areas.
nearest gives the index of the nearest theta-phi in the given theta-phi file."""

"""sens_source1 = SkyCoord(phideg1*unit.deg,(thetadeg1-90)*unit.deg,frame='gcrs')
#sens_source2 = SkyCoord(dec=(theta2-np.pi/2)*unit.rad,ra = phi2*unit.rad,distance=1*unit.m,frame='gcrs')
sens_sat = SkyCoord(np.transpose(sensfilethetaphi)[1]*unit.rad,(np.transpose(sensfilethetaphi)[0]-np.pi/2)*unit.rad,frame='gcrs')
nearest1 = astropy.coordinates.match_coordinates_sky(sens_source1, sens_sat, nthneighbor=1, storekdtree='kdtree_sky')[0]
#nearest2 = astropy.coordinates.match_coordinates_sky(sens_source2, sens_sat, nthneighbor=1, storekdtree='kdtree_sky')[0]"""

"sens_source1 = SkyCoord(phideg1*unit.deg,(thetadeg1-90)*unit.deg,frame='gcrs')\n#sens_source2 = SkyCoord(dec=(theta2-np.pi/2)*unit.rad,ra = phi2*unit.rad,distance=1*unit.m,frame='gcrs')\nsens_sat = SkyCoord(np.transpose(sensfilethetaphi)[1]*unit.rad,(np.transpose(sensfilethetaphi)[0]-np.pi/2)*unit.rad,frame='gcrs')\nnearest1 = astropy.coordinates.match_coordinates_sky(sens_source1, sens_sat, nthneighbor=1, storekdtree='kdtree_sky')[0]\n#nearest2 = astropy.coordinates.match_coordinates_sky(sens_source2, sens_sat, nthneighbor=1, storekdtree='kdtree_sky')[0]"

In [36]:
matchcoord = SkyCoord(phideg1*unit.degree,(thetadeg1-90)*unit.degree)
catalogcoord = SkyCoord(np.rad2deg(np.transpose(sensfilethetaphi)[1])*unit.degree,(np.rad2deg(np.transpose(sensfilethetaphi)[0]-np.pi/2))*unit.degree)
        

In [37]:
"""astropy.coordinates.match_coordinates_sky:
Finds the nearest on-sky matches of a coordinate or coordinates in a set of catalog coordinates.
This finds the on-sky closest neighbor, which is only different from the 3-dimensional match if distance 
is set in either matchcoord or catalogcoord."""
index = astropy.coordinates.match_coordinates_sky(matchcoord, catalogcoord, nthneighbor=1, storekdtree='kdtree_sky')[0]

In [38]:
#index[2],np.deg2rad(phideg1[2]),np.deg2rad((thetadeg1)[2]),(np.transpose(sensfilethetaphi)[1])[830]

In [39]:
sensitivitydata1 = []
#sensitivitydata2 = []
sensfilethetaphi = []
for i in range(len(sensfile)):
    sensfilethetaphi.append((sensfile[i][0], sensfile[i][1]))
    

In [40]:
for i in range(len(theta)):
    sensitivitydata1.append(sensfile[index[i]][3])


In [41]:
"""replacing nan by 0"""
for k in range(len(sensitivitydata1)):
    if np.isnan(sensitivitydata1[k])==True:
        sensitivitydata1[k] = 0
        
sensdatabackup1 = sensitivitydata1
sensitivitydata2 = sensitivitydata1

In [43]:
%%time
numtrig = 0
for i in range(len(lumino)):
    if lumino[i]>0:numtrig = numtrig + 1
line1=[] #DD
line2=[] #LIGO
line3=[] #DD + LIGO
line4=[] #D : chosen without any reason as the first among these two (ephem = astrosat1)
line5=[] #D + LIGO
line6=[] #DD detections which LIGO didn't see
line7=[] #D detections which LIGO didn't see
print(numtrig)
# senssattemp = 1.4*10**(-7)
# senssattemp = 7.0*10**(-8)

#steps in is_visible calc:
# skycoord object of source
# skycoord object of earth


for c_snr in [8,12]:
    snr=c_snr
    print('\nSNR =', snr)
    boolvis = []
    countsaa1, countsens1, countearth1, countvis1= 0,0,0,0
    countsaa2, countsens2, countearth2, countvis2 = 0,0,0,0
    count1sat,count1sat = 0,0
    count2sat,count2sat = 0,0
    count0sat,count0sat = 0,0
    vis1, vis2 = 0,0
    satd0, satd1, satd2 = 0,0,0
    dnol, ddnol0, ddnol1,ddnol2 = 0,0,0,0
    lowsnr = 0
    for i in range(len(homotime)):
        if i%10000==0:
            print(int(i*100/len(homotime)),'%',end='\r', flush=True)
        if snrLHVKI[i]<snr:
                lowsnr=lowsnr + 1
        if lumino[i]>0:
            satyes, yes, var3 = 0,0,0
            satvisible = is_visible_bool[i]
            if is_saa(sublat1[i],sublong1[i])==True:
                countsaa1=countsaa1 + 1
            if sensitivitydata1[i]>lumino[i]:
                countsens1=countsens1+1
            if satvisible==False:
                countearth1 = countearth1 +1
            if is_saa(sublat1[i],sublong1[i])==False and snrLHVKI[i]>=snr and sensitivitydata1[i]<=lumino[i] and satvisible==True:
                countvis1 = countvis1+1
                satyes = satyes + 1
            if is_saa(sublat1[i],sublong1[i])==False and sensitivitydata1[i]<=lumino[i] and satvisible==True:
                vis1 = vis1+1
                yes = yes + 1
            if is_saa(sublat1[i],sublong1[i])==False and snrLHVKI[i]<snr and sensitivitydata1[i]<=lumino[i] and satvisible==True:
                var3=var3+1
                dnol=dnol+1
                
            satvisible2 = is_visible_bool2[i]

            if is_saa(sublat2[i],sublong2[i])==True:
                countsaa2=countsaa2 + 1
            if sensitivitydata2[i]>lumino[i]:
                countsens2=countsens2+1
            if satvisible2==False:
                countearth2 = countearth2 +1
            if is_saa(sublat2[i],sublong2[i])==False and snrLHVKI[i]>=snr and sensitivitydata2[i]<=lumino[i] and satvisible2==True:
                countvis2 = countvis2+1
                satyes = satyes + 1
            if is_saa(sublat2[i],sublong2[i])==False and sensitivitydata2[i]<=lumino[i] and satvisible2==True:
                vis2 = vis2+1
                yes = yes + 1           
            if is_saa(sublat2[i],sublong2[i])==False and snrLHVKI[i]<snr and sensitivitydata2[i]<=lumino[i] and satvisible2==True:
                var3=var3+1

            if satyes==0:
                count0sat=count0sat+1
            if satyes==1:
                count1sat=count1sat+1
            if satyes==2:
                count2sat=count2sat+1
            if yes==0:
                satd0=satd0+1
            if yes==1:
                satd1=satd1+1
            if yes==2:
                satd2=satd2+1
            if var3==0:
                ddnol0=ddnol0+1
            if var3==1:
                ddnol1=ddnol1+1
            if var3==2:
                ddnol2=ddnol2+1
    
    print('***100% Completed***\n',end='\r',flush=True)        
    print('lowsnr', lowsnr,'\t', lowsnr*100/numtrig)
    print('goodsnr', numtrig - lowsnr,'\t', (numtrig - lowsnr)*100/numtrig)
    print('\nDL')
    print('countvis1', countvis1,'\t', countvis1*100/(numtrig-lowsnr))
    print('countvis2', countvis2,'\t', countvis2*100/(numtrig-lowsnr))
    print('oD')
    print('vis1', vis1,'\t', vis1*100/(numtrig))
    print('vis2', vis2,'\t', vis2*100/(numtrig))
    print('\nDL', count0sat, count1sat, count2sat)
    print('oD', satd0, satd1, satd2)
    print((count1sat+count2sat+count0sat)*100/numtrig, '-- should be equal to 100%\n')
    print((count1sat+count2sat), '\t',(count1sat+count2sat)*100/(numtrig-lowsnr) ,'(' + str(count1sat*100/numtrig+count2sat*100/numtrig)+ ')')
    print('Line 1 element(Seen by D-Daksha)                      ', satd1+satd2)
    print('Line 2 element(Seen by Ligo network)                  ', len(homotime)-lowsnr)
    print('Line 3 element(Seen by both D-Daksha & LIGO)          ', count1sat+count2sat)
    print('Line 4 element(Seen by only one Daksha)               ', vis1)
    print('Line 5 element(Seen by both Daksha & LIGO)            ', countvis1)
    print('Line 6 element(LIGO missed which D-Dakhsa detected)   ', ddnol1+ddnol2)
    print('Line 7 element(LIGO missed which Daksha detected)     ', dnol)
    line1.append(satd1+satd2)
    line2.append(len(homotime)-lowsnr)
    line3.append(count1sat+count2sat)
    line4.append(vis1)
    line5.append(countvis1)
    line6.append(ddnol1+ddnol2)
    line7.append(dnol)

1000

SNR = 8
***100% Completed***
lowsnr 813 	 81.3
goodsnr 187 	 18.7

DL
countvis1 132 	 70.58823529411765
countvis2 125 	 66.84491978609626
oD
vis1 707 	 70.7
vis2 695 	 69.5

DL 815 113 72
oD 5 588 407
100.0 -- should be equal to 100%

185 	 98.93048128342247 (18.5)
Line 1 element(Seen by D-Daksha)                       995
Line 2 element(Seen by Ligo network)                   187
Line 3 element(Seen by both D-Daksha & LIGO)           185
Line 4 element(Seen by only one Daksha)                707
Line 5 element(Seen by both Daksha & LIGO)             132
Line 6 element(LIGO missed which D-Dakhsa detected)    810
Line 7 element(LIGO missed which Daksha detected)      575

SNR = 12
***100% Completed***
lowsnr 934 	 93.4
goodsnr 66 	 6.6

DL
countvis1 48 	 72.72727272727273
countvis2 37 	 56.06060606060606
oD
vis1 707 	 70.7
vis2 695 	 69.5

DL 936 43 21
oD 5 588 407
100.0 -- should be equal to 100%

64 	 96.96969696969697 (6.4)
Line 1 element(Seen by D-Daksha)                      